In [1]:
import time
import torch
from torch.distributions import Categorical, kl
import numpy as np
from tqdm import tqdm
# from d2l.torch import Animator

from net import Net
from aco import ACO
from faco import FACO
from utils import gen_pyg_data, load_test_dataset

torch.manual_seed(12345)

EPS = 1e-10
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print(f'Using device: {device}')

Using device: cuda:0


In [2]:
from xml.parsers.expat import model


@torch.no_grad()
def infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, k_sparse=None):
    heu_mat = None
    if model is not None:
        model.eval()
        heu_vec = model(pyg_data)
        heu_mat = model.reshape(pyg_data, heu_vec) + EPS
        heu_mat = heu_mat.cpu()

    aco = FACO(
        distances.cpu(),
        n_ants,
        heuristic=heu_mat.cpu() if heu_mat is not None else heu_mat,
        k_sparse=k_sparse,
        local_search_type="2opt",
    )
    results = torch.zeros(size=(len(t_aco_diff),))
    diversities = torch.zeros(size=(len(t_aco_diff),))
    elapsed_time = 0
    for i, t in enumerate(t_aco_diff):
        results[i], diversities[i], t = aco.run(t)
        elapsed_time += t
    return results, diversities, elapsed_time
        
    
@torch.no_grad()
def test(dataset, model, n_ants, t_aco, k_sparse=None):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i + 1] - _t_aco[i] for i in range(len(_t_aco) - 1)]
    instance_costs = []
    instance_divs = []
    instance_times = []
    bar = tqdm(dataset, dynamic_ncols=True)
    for pyg_data, distances in bar:
        results, diversities, elapsed_time = infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, k_sparse)
        instance_costs.append(results)
        instance_divs.append(diversities)
        instance_times.append(elapsed_time)
        bar.set_description(f'Avg instance costs: {results.mean().numpy()}, time: {elapsed_time:.2f}s')
    instance_costs = torch.stack(instance_costs)
    instance_divs = torch.stack(instance_divs)
    instance_times = np.array(instance_times)
    # Return per-instance averages and overall averages
    return instance_costs, instance_divs, instance_times, torch.mean(instance_costs, dim=0), torch.mean(instance_divs, dim=0), np.mean(instance_times)

### Test on TSP20

DeepACO

In [3]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10]
test_list = load_test_dataset(n_node, k_sparse, device)
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp/tsp{n_node}.pt', map_location=device))
instance_costs, instance_divs, instance_times, avg_aco_best, avg_aco_divs, avg_aco_time = test(test_list, net_tsp, n_ants, t_aco, k_sparse)
print('avg duration: ', avg_aco_time)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

Avg instance costs: 3.4748950004577637, time: 0.04s: 100%|██████████| 1280/1280 [02:00<00:00, 10.64it/s]

avg duration:  0.06306397076696157
T=1, average cost is 3.8431544303894043.
T=10, average cost is 3.842236280441284.


ACO

In [4]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10]
test_list = load_test_dataset(n_node, k_sparse, device)
instance_costs, instance_divs, instance_times, avg_aco_best, avg_aco_divs, avg_aco_time = test(test_list, net_tsp, n_ants, t_aco, k_sparse)
print('avg duration: ', avg_aco_time)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

Avg instance costs: 4.3400468826293945, time: 0.04s:  35%|███▍      | 442/1280 [00:26<00:50, 16.52it/s]


KeyboardInterrupt: 